#__Video Classification Using Hybrid Model__
Let's see how to classify the video using transfer learning and a recurrent model on the UCF101 dataset.


## Steps to be followed:
1. Download data and import the required libraries
2. Read the data from datasets and print the ten rows
3. Define the functions for cropping and loading video frames
4. Build a feature extraction model using the InceptionV3 architecture
5. Create a string lookup table for labels and print the vocabulary of the label processor
6. Prepare video data for training and testing by extracting frame features
7. Define and train a sequence model using GRU layers
8. Load a test video, extract frame features, and make predictions using the sequence model

### Step 1: Download data and import the required libraries
- Download the dataset.
- Import the required libraries.

In [1]:
!wget -q --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
!wget -q --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip

In [2]:
%%capture
!unrar e UCF101.rar data/
!unzip -qq UCF101TrainTestSplits-RecognitionTask.zip

In [4]:
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
import shutil

Open the __.txt__ file which has the names of the training videos.

Create a DataFrame having video names.


In [5]:
f = open("ucfTrainTestlist/trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

video_name
0  ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1  ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2  ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3  ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4  ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1

Open the __.txt__ file which has the names of the test videos.

Create a DataFrame having video names.

In [6]:
with open("ucfTrainTestlist/testlist01.txt", "r") as f:
    temp = f.read()
videos = temp.split("\n")

test = pd.DataFrame()
test["video_name"] = videos
test = test[:-1]
test.head()

video_name
0  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi

In [7]:
print("Length of train data: ", len(train))
print("Length of test data: ", len(test))

Length of train data:  9537
Length of test data:  3783


- Define the __extract_tag__ function that extracts a tag from the video path. This is done by splitting the video path by '/' and returning the first part.
- Define the __separate_video_name__ function, which separates the video name from the video path. This is achieved by splitting the video name by / and returning the second part.
- Define the __rectify_video_name__ function to rectify the video name by splitting the video name by " " and returning the first part.
- Define the __move_videos__ function:
   - Check if the output directory exists. If not, create the directory using __os.mkdir__.
   - Iterate over the DataFrame, __df__, using a progress bar from the __tqdm__ library.
   - For each row in the DataFrame, extract the video file name from the __video_name__ column, create its path, and then copy the video file to the output directory using __shutil.copy2__.
   - After the loop ends, print the total number of videos in the output directory.


In [8]:
# Define a function to extract the tag from a video path
def extract_tag(video_path):
    # Split the video path by "/" and return the first part (the tag)
    return video_path.split("/")[0]

# Define a function to separate the video name from a full path
def separate_video_name(video_name):
    # Split the video name by "/" and return the second part (the video name)
    return video_name.split("/")[1]

# Define a function to rectify the video name by removing any extra information
def rectify_video_name(video_name):
    # Split the video name by space " " and return the first part (the rectified video name)
    return video_name.split(" ")[0]

# Define a function to move videos from a source directory to an output directory based on a DataFrame
def move_videos(df, output_dir):
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for i in tqdm(range(df.shape[0])):
        videoFile = df['video_name'][i].split("/")[-1]
        videoPath = os.path.join("data", videoFile)
        shutil.copy2(videoPath, output_dir)
    print()
    print(f"Total videos: {len(os.listdir(output_dir))}")

### Step 2: Read the data from datasets and print the ten rows

- Define the values of parameters **IMG_SIZE**, **BATCH_SIZE**, **EPOCHS**, **MAX_SEQ_LENGTH**, and **NUM_FEATURES.**


In [9]:
# Set the image size to 224 pixels (usually used in image processing tasks)
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 2

# Define the maximum sequence length (used in natural language processing tasks)
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

### Step 3: Define the functions for cropping and loading video frames
- Define a function named **crop_center_square** that crops a frame to a square shape by determining the minimum dimension and calculating the starting coordinates.
- Define a function named __load_video__ that loads a video file, crops each frame to a square shape, resizes it, and converts the color channels.
- Open the video file using **cv2.VideoCapture** and initialize an empty list called frames.
- Read frames from the video, crop them to a square shape, resize them, convert the color channels, and append them to the frames list.
- If the maximum number of frames is reached or the video ends, exit the loop.
- Release the video capture.
- Convert the frame list to a NumPy array.
- Return the array of frames.

In [10]:
# Define a function to crop the center square of a frame
def crop_center_square(frame):
    y, x = frame.shape[0:2] # Get the height and width of the frame
    min_dim = min(y, x) # Calculate the minimum dimension (height or width)
    start_x = (x // 2) - (min_dim // 2) # Calculate the starting x-coordinate for cropping
    start_y = (y // 2) - (min_dim // 2) # Calculate the starting y-coordinate for cropping
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim] # Crop the center square and return it

# Define a function to load a video from a file
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)  # Open the video file for reading
    frames = [] # Create an empty list to store video frames
    try:
        while True:
            ret, frame = cap.read() # Read the next frame from the video
            if not ret:
                break # If the frame couldn't be read, break out of the loop
            frame = crop_center_square(frame) # Crop the frame to a square shape
            frame = cv2.resize(frame, resize) # Resize the frame to the specified dimensions
            frame = frame[:, :, [2, 1, 0]] # Reorder the color channels from BGR to RGB
            frames.append(frame) # Append the processed frame to the list of frames

            if len(frames) == max_frames:
                break # If the desired number of frames is reached, break out of the loop
    finally:
        cap.release() # Release the video capture object when finished reading the video file
    return np.array(frames) # Convert the list of frames to a NumPy array and return it

__Observation:__
- The code defines two functions, **crop_center_square** and __load_video__, which can be used to crop frames from videos and load videos as arrays of frames, respectively.

### Step 4: Build a feature extraction model using InceptionV3 architecture

- Create a feature extractor using the InceptionV3 model from keras.applications with specific configurations.
- Assign the preprocess_input function from **keras.applications.inception_v3** to the **variable preprocess_input**.
- Create an input layer with the shape __(IMG_SIZE, IMG_SIZE, 3)__ using **keras.Input**.
- Preprocess the input using the **preprocess_input** function.
- Pass the preprocessed input through the feature extractor to obtain the outputs.
- Create a model with the inputs and outputs using **keras.Model** and assign it to the variable **feature_extractor**.

In [11]:
# Define a function to build a feature extractor model using InceptionV3 architecture
def build_feature_extractor():
    # Create an InceptionV3 model pre-trained on ImageNet data
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet", # Use pre-trained weights from ImageNet
        include_top=False, # Exclude the top (classification) layer
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3), # Define the input shape
    )

    # Import the preprocessing function for InceptionV3
    preprocess_input = keras.applications.inception_v3.preprocess_input

    # Define an input layer for the model with the specified input shape
    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))

    # Apply preprocessing to the input data using the imported function
    preprocessed = preprocess_input(inputs)

    # Pass the preprocessed input through the feature extractor model
    outputs = feature_extractor(preprocessed)

    # Create and return a Keras Model with inputs and outputs
    return keras.Model(inputs, outputs, name="feature_extractor")

# Call the build_feature_extractor function to create the feature extractor model
feature_extractor = build_feature_extractor()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


__Observations:__
- The code defines a function **build_feature_extractor** that creates a feature extractor model using InceptionV3 architecture.
- The model takes inputs of size __(IMG_SIZE, IMG_SIZE, 3)__, preprocesses the inputs, and produces the outputs.
- The created feature extractor model is assigned to the variable **feature_extractor**.

In [12]:
# This line adds a new column named "tag" to the DataFrame "train."
train["tag"] = train["video_name"].apply(extract_tag)

# It applies the "separate_video_name" function to each element in the "video_name" column.
train["video_name"] = train["video_name"].apply(separate_video_name)
train.head()

video_name             tag
0  v_ApplyEyeMakeup_g08_c01.avi 1  ApplyEyeMakeup
1  v_ApplyEyeMakeup_g08_c02.avi 1  ApplyEyeMakeup
2  v_ApplyEyeMakeup_g08_c03.avi 1  ApplyEyeMakeup
3  v_ApplyEyeMakeup_g08_c04.avi 1  ApplyEyeMakeup
4  v_ApplyEyeMakeup_g08_c05.avi 1  ApplyEyeMakeup

In [13]:
# Assign the result of applying the 'rectify_video_name' function to the 'video_name' column in the 'train' DataFrame.
train["video_name"] = train["video_name"].apply(rectify_video_name)
train.head()

video_name             tag
0  v_ApplyEyeMakeup_g08_c01.avi  ApplyEyeMakeup
1  v_ApplyEyeMakeup_g08_c02.avi  ApplyEyeMakeup
2  v_ApplyEyeMakeup_g08_c03.avi  ApplyEyeMakeup
3  v_ApplyEyeMakeup_g08_c04.avi  ApplyEyeMakeup
4  v_ApplyEyeMakeup_g08_c05.avi  ApplyEyeMakeup

In [14]:
# Assign a new column 'tag' to the DataFrame 'test' by applying the 'extract_tag' function to each value in the 'video_name' column.
test["tag"] = test["video_name"].apply(extract_tag)

test["video_name"] = test["video_name"].apply(separate_video_name)
test.head()

video_name             tag
0  v_ApplyEyeMakeup_g01_c01.avi  ApplyEyeMakeup
1  v_ApplyEyeMakeup_g01_c02.avi  ApplyEyeMakeup
2  v_ApplyEyeMakeup_g01_c03.avi  ApplyEyeMakeup
3  v_ApplyEyeMakeup_g01_c04.avi  ApplyEyeMakeup
4  v_ApplyEyeMakeup_g01_c05.avi  ApplyEyeMakeup

**Filter Top-N Activities**

In [15]:
# Set n to 10, specifying the number of top categories to select
n = 10

# Calculate the value counts of the "tag" column in the "train" DataFrame and select the n largest counts.
topNActs = train["tag"].value_counts().nlargest(n).reset_index()["tag"].tolist()

# Create a new DataFrame "train_new" by selecting rows from the "train" DataFrame
# where the "tag" column is in the list of the top n categories.
train_new = train[train["tag"].isin(topNActs)]

test_new = test[test["tag"].isin(topNActs)]

train_new.shape, test_new.shape

((1171, 2), (459, 2))

**Observation:**
- The output **((1171, 2), (459, 2))** is a tuple showing the shapes of **train_new** and **test_new**. The **train_new** DataFrame has 1171 rows and 2 columns, and the **test_new** DataFrame has 459 rows and 2 columns.

In [16]:
train_new = train_new.reset_index(drop=True)
test_new = test_new.reset_index(drop=True)

### Step 5: Create a string lookup table for labels and print the vocabulary of the label processor
- Create a label processor using **keras.layers.StringLookup.**
- Set the number of out-of-vocabulary (OOV) indices to **0.**
- Set the vocabulary of the label processor to the unique values from the **tag** column of the **train_df** DataFrame.
- Retrieve the vocabulary of the label processor using **label_processor.get_vocabulary().**

In [17]:
# Create a StringLookup layer for mapping labels to integers
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train["tag"])  # Vocabulary is the unique tags in the "train" dataset
)
# Print the vocabulary of the label processor
print(label_processor.get_vocabulary())

[np.str_('ApplyEyeMakeup'), np.str_('ApplyLipstick'), np.str_('Archery'), np.str_('BabyCrawling'), np.str_('BalanceBeam'), np.str_('BandMarching'), np.str_('BaseballPitch'), np.str_('Basketball'), np.str_('BasketballDunk'), np.str_('BenchPress'), np.str_('Biking'), np.str_('Billiards'), np.str_('BlowDryHair'), np.str_('BlowingCandles'), np.str_('BodyWeightSquats'), np.str_('Bowling'), np.str_('BoxingPunchingBag'), np.str_('BoxingSpeedBag'), np.str_('BreastStroke'), np.str_('BrushingTeeth'), np.str_('CleanAndJerk'), np.str_('CliffDiving'), np.str_('CricketBowling'), np.str_('CricketShot'), np.str_('CuttingInKitchen'), np.str_('Diving'), np.str_('Drumming'), np.str_('Fencing'), np.str_('FieldHockeyPenalty'), np.str_('FloorGymnastics'), np.str_('FrisbeeCatch'), np.str_('FrontCrawl'), np.str_('GolfSwing'), np.str_('Haircut'), np.str_('HammerThrow'), np.str_('Hammering'), np.str_('HandstandPushups'), np.str_('HandstandWalking'), np.str_('HeadMassage'), np.str_('HighJump'), np.str_('HorseRac

__Observations:__
- The code creates a label processor that maps labels from text to integer indices.
- It uses the unique values from the **tag** column of the **train** DataFrame as the vocabulary for the label processor.
- The output is the vocabulary of the label processor, which is a list of unique labels.

### Step 6: Prepare video data for training and testing by extracting frame features
- Define a function named **prepare_all_videos** that inputs a DataFrame (df) and a root directory **(root_dir)**.
- Retrieve the video paths and labels from the DataFrame and encode the labels using **label_processor**.
- Initialize arrays to store frame masks and frame features for each video.
- Iterate over each video in the dataset, load the frames, and extract features using the **feature_extractor** model.
- Update the arrays with the extracted features and masks for each video.
- Call the **prepare_all_videos** function on the train and test DataFrames, storing the returned values in **train_data**, **train_labels**, **test_data**, and __test_labels__.
- Finally, print the shape of the frame features in the train set and the shape of the frame masks in the train set.

In [18]:
# Define a function to prepare video data
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    for idx, path in enumerate(video_paths):
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train, "train")
test_data, test_labels = prepare_all_videos(test, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Frame features in train set: (9537, 20, 2048)
Frame masks in train set: (9537, 20)


__Observations:__
- The code processes videos by extracting frame features and creating frame masks.
- It then returns the frame features, frame masks, and labels for the train and test sets.
- The output is the shape of the frame features in the train set and the shape of the frame masks in the train set.

### Step 7: Define and train a sequence model using GRU layers
- Define a function named **get_sequence_model** that creates a sequence model for video classification.
- Create input layers for frame features and masks.
- Apply two GRU layers to the frame features input, with the second GRU layer returning only the last output.
- Add a dropout layer, a dense layer with ReLU activation, and a final dense layer with softmax activation for the output.
- Compile the model with sparse categorical cross-entropy loss, Adam optimizer, and accuracy metric.
- Define a function named __run_experiment__ for running the training and evaluation.
- Set up a checkpoint to save the best model during training.
- Create the sequence model using **get_sequence_model.**
- Train the model on the training data with a validation split, a specified number of epochs, and the checkpoint callback.
- Load the best weights saved during training.
- Evaluate the model on the test data and print the test accuracy.
- Return the history object and the trained sequence model.
- Call the **run_experiment** function and store the returned values in __(history)__ and **sequence_model.**


In [22]:
# Define a function to create a sequence model
def get_sequence_model():
    # Get the vocabulary of class labels
    class_vocab = label_processor.get_vocabulary()

    # Define input layers for frame features and mask
    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Apply a GRU (Gated Recurrent Unit) layer with 16 units and return sequences
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )

    # Apply another GRU layer with 8 units
    x = keras.layers.GRU(8)(x)
    # Apply dropout to the output of the GRU layer
    x = keras.layers.Dropout(0.4)(x)
    # Apply a Dense layer with 8 units and 'relu' activation
    x = keras.layers.Dense(8, activation="relu")(x)
    # Apply a Dense output layer with softmax activation for class probabilities
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    # Create the RNN model with input and output layers
    rnn_model = keras.Model([frame_features_input, mask_input], output)

    # Compile the model with loss, optimizer, and metrics
    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

# Define a function to run the experiment
def run_experiment():
    # Define a file path to save model checkpoints
    filepath = "/tmp/video_classifier.weights.h5"

    # Create a callback to save the best model weights during training
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    # Get the sequence model
    seq_model = get_sequence_model()

    # Train the model using training data and labels with validation split
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    # Load the best model weights
    seq_model.load_weights(filepath)

    # Evaluate the model on test data and calculate test accuracy
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/2
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0161 - loss: 4.5974
Epoch 1: val_loss improved from inf to 4.78488, saving model to /tmp/video_classifier.weights.h5
209/209 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - accuracy: 0.0161 - loss: 4.5973 - val_accuracy: 0.0000e+00 - val_loss: 4.7849
Epoch 2/2
208/209 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.0177 - loss: 4.5321
Epoch 2: val_loss did not improve from 4.78488
209/209 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.0177 - loss: 4.5320 - val_accuracy: 0.0000e+00 - val_loss: 4.9503
119/119 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.0187 - loss: 4.5587
Test accuracy: 1.3%


__Observations:__
- Training progress and validation metrics will be displayed during the model training process.
- After training, the model will be evaluated on the test data, and the test accuracy will be printed.

### Step 8: Load a test video, extract frame features, and make predictions using the sequence model
- Load a random test video path.
- Call the **sequence_prediction** function with the test video path.
- Within the **sequence_prediction** function:

  a. Get the vocabulary of the classes.

  b. Load the frames of the video.

  c. Prepare the frames for sequence prediction by extracting frame features and creating a frame mask.

  d. Use the trained sequence model to predict the probabilities of each class in the video.

  e. Print the predicted class probabilities in descending order.

- Assign the frames of the test video to the variable test_frames.

In [23]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

In [24]:
test_video = np.random.choice(test["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)


Test video path: v_BoxingPunchingBag_g01_c01.avi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step
  CricketShot:  1.09%
  PlayingCello:  1.09%
  PlayingDhol:  1.09%
  HorseRiding:  1.09%
  BoxingPunchingBag:  1.09%
  PlayingGuitar:  1.08%
  BenchPress:  1.08%
  IceDancing:  1.08%
  HeadMassage:  1.08%
  BandMarching:  1.08%
  Hammering:  1.08%
  Bowling:  1.08%
  HammerThrow:  1.08%
  Drumming:  1.08%
  JumpRope:  1.08%
  BaseballPitch:  1.08%
  PlayingFlute:  1.07%
  Diving:  1.07%
  PoleVault:  1.07%
  PlayingDaf:  1.07%
  Kayaking:  1.07%
  PlayingSitar:  1.07%
  Billiards:  1.07%
  Archery:  1.07%
  Haircut:  1.07%
  CricketBowling:  1.07%
  FrontCrawl:  1.07%
  GolfSwing:  1.07%
  Nunchucks:  1.06%
  Basketball:  1.06%
  BabyCrawling:  1.06%
  HandstandPushups:  1.06%
  CliffDiving:  1.06%
  ApplyEyeMakeup:  1.06%
  BoxingSpeedBag:  1.06%
  Biking:  1.06%
  BasketballDunk:  1.06%
  FloorGymnastics:  1.06%
  BrushingTeeth:  1.06%
  Mixing:  1.06%
  BlowDryHair:  1.06%
  PommelHorse:  1.06%
 

__Observations:__
- The test video path will be printed.
- The predicted class probabilities for the test video will be printed, showing the class label and the corresponding probability.
- The frames of the test video will be assigned to the **test_frames** variable.